# 5. 소프트맥스 회귀(Softmax Regression)
## 5-1. 원-핫 인코딩(One-Hot Encoding)
선택해야 하는 선택지의 개수만큼의 차원을 가지면서, 각 선택지의 인덱스에 해당하는 원소에는 1, 나머지 원소에는 0의 값을 가지도록 하는 표현 방법

## 5-2. 소프트맥스 회귀(Softmax Regression) 이해하기
로지스틱 회귀 - 이진분류 / 소프트맥스 회귀 - 다중 클래스 분류 <br/>

### 1. 다중 클래스 분류
#### 1. 로지스틱 회귀
시그모이드 함수로 예측값을 0과 1 사이의 값으로 만듦. <br/>
가설: H(X) = sigmoid(WX + B) <br/>
#### 2. 소프트맥스 회귀
확률의 총 합이 1이 되도록, 각 클래스마다 소수 확률을 할당함. <br/>
가설: H(X) = softmax(WX + B) <br/>

### 2. 소프트맥스 함수(Softmax function)
k차원의 벡터를 입력받아 각 클래스에 대한 확률을 추정함
### 3. 붓꽃 품종 분류하기 행렬 연산으로 이해하기
### 4. 비용 함수(Cost function)
소프트맥스 회귀에서는 비용함수로 크로스 엔트로피 함수 사용 <br/>
로지스틱 회귀에서의 크로스 엔트로피 함수식과 동일한데 2 대신 k가 들어감

## 5-3. 소프트맥스 회귀의 비용 함수 구현하기

In [1]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

### 1. 파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨)

In [3]:
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim=0)  #3개의 원소의 값이 0~1 사이의 값을 가지는 벡터로 변환됨
print(hypothesis)
hypothesis.sum()  #변환된 벡터들의 합은 1

tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

In [4]:
#비용함수 직접 구현
z = torch.rand(3, 5, requires_grad=True) #행 개수 3, 최대값 5

hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [5]:
# 각 샘플에 대해 임의의 레이블(정답) 만듦
y = torch.randint(5, (3,)).long()  #최대값 5, 크기 3(tuple)
print(y)

tensor([0, 2, 1])


In [9]:
# 각 레이블에 대해 원-핫 인코딩 수행
y_one_hot = torch.zeros_like(hypothesis)  # hypothesis와 같은 형태의 3X5 텐서 생성
y_one_hot.scatter_(1, y.unsqueeze(1), 1)  # 첫번째 인자로 dim=1에 대해 수행
# 세번째 인자로 두번째 인자인 y_unsqueeze(1)이 알려주는 위치에 숫자 1을 넣도록 함.

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [10]:
print(y.unsqueeze(1)) #(3,)의 크기를 가졌던 y 텐서는 (3x1)텐서가 됨. 특정 위치에 1인 차원을 추가한다
print(y_one_hot)

tensor([[0],
        [2],
        [1]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [11]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


### 2. 파이토치로 소프트맥스의 비용 함수 구현하기 (하이-레벨)

In [12]:
# 1. F.softmax() = torch.log() = F.log_softmax() #소프트맥스 함수
#Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [13]:
#High level
F.log_softmax(z, dim=1) #torch.log(F.softmax())와 F.log_softmax()와 동일한 결과

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [18]:
# 2. F.log_softmax() + F.nll_loss() = F.cross_entropy() #비용 함수
#Low level
#첫번째 수식
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [19]:
#두번째 수식
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [20]:
#High level
#세번째 수식
F.nll_loss(F.log_softmax(z, dim=1), y)  #nll = Negative Log Likelihood

tensor(1.4689, grad_fn=<NllLossBackward>)

In [21]:
#네번째 수식
F.cross_entropy(z, y)  #비용 함수에 소프트맥스 함수까지 포함되어 있음

tensor(1.4689, grad_fn=<NllLossBackward>)